In [1]:
import newspaper
import pandas as pd
from datetime import datetime
from pygooglenews import GoogleNews

import warnings
warnings.filterwarnings("ignore")

In [2]:
dates = pd.date_range(start="2022-01-01",end="2022-07-31").astype('str')
data = pd.DataFrame(columns=['title', 'text', 'published_date'])
logs = pd.DataFrame(columns=['start', 'end', 'total_links'])

# data = pd.read_csv('data.csv')
# logs = pd.read_csv('logs.csv')

with open('proxies.txt', 'r') as f:
    proxies = f.read().splitlines() 

In [3]:
def get_links(start, end, query):
    try:

        response = gn.search(query, from_ = start, to_ = end)
        return response['entries']
    except Exception as exp:
        for proxy in proxies:
            try:
                response = gn.search(query, from_=start, to_=end, proxies=proxy)
                return response['entries']
            except:
                pass

In [4]:
def get_article(link):
    try:
        article = newspaper.Article(url=link, languge='en')
        article.download()
        article.parse()
        
        return {
            "title": str(article.title),
            "text": str(article.text),
            "published_date": str(article.publish_date)
        }
    except:
        pass

In [ ]:
for country in ['AU', 'US', 'UK', 'FR', 'CA', 'DE', 'IE', 'NZ']:
    gn = GoogleNews(lang='en', country=country) 
    for i in range(0, len(dates), 2):
        links = [entry['link'] for query in ['Business News', 'Sports News', 'Entertainment', 'Worlds News', 'Nation News',
                                             'Technology News', 'Entertainment News', 'Science News', 'Health News']
                                             for entry in get_links(dates[i], dates[i+1], query)]
        
        print('On country: {ctry}'.format(ctry=country))
        print({'start': dates[i],'end': dates[i+1], 'total_links': len(links)})
        logs = logs.append({'start': dates[i],'end': dates[i+1], 'total_links': len(links)}, ignore_index=True)
        
        articles = list(map(get_article, links))
        Not_none_values = filter(None.__ne__, articles)
        articles = list(Not_none_values)
        print("Articles recieved: "+str(len(articles)))
        data = data.append(pd.DataFrame(articles), ignore_index=True)

        logs.to_csv('logs.csv', index=False)
        data.to_csv('data.csv', index=False)

On country: AU
{'start': '2022-01-01', 'end': '2022-01-02', 'total_links': 114}
